In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
def read_and_decode_mcge_p(path,file):
    with open(os.path.join(path,file), 'r') as f:
        content = f.read()
    number = []
    for s in content.split():
        try:
            x = float(s)
            number.append(x)
        except:
            pass
    content_matrix = np.reshape(number, (8, 5, 8))
    num_proc = content_matrix[0, 0, 2]
    average_time = content_matrix[:,:,3]
    comm_time = content_matrix[:,:,5]
    dist_time = content_matrix[:,:,7]
    return num_proc, average_time, comm_time, dist_time

def read_and_decode_ge_scalapack(path, file):
    with open(os.path.join(path,file), 'r') as f:
        content = f.read()
    content_list = content.split()
    content_2 = content_list[9::9]
    number = []
    for content_2_element in content_2:
        for s in content_2_element.split(','):
            try:
                x = float(s)
                number.append(x)
            except:
                pass
    content_matrix = np.reshape(number, (8, 5, 6))
    average_time = content_matrix[:, :, 5]
    tmp_number = content_list[8].split(',')
    num_proc = int(tmp_number[0])
    return num_proc, average_time



def generate_metrics_mcge_p(path):
    '''
    this function will generate 3x8x8x5 matrices for mc_p and ge_p
    1st dimension: 0-total execution time  1-total communication time 2-data distribution time
    2nd dimension:problem size in thousand (1-8K)
    3rd dimension: number of processors-1 2 4 8 16 32 64 128
    4th dimension: different independent runs'''
    
    #initialize result
    result = np.zeros((3,8,8,5), dtype = float)
    file_list = os.listdir(path)
    for file in file_list:
        num_proc, average_time, comm_time, dist_time = read_and_decode_mcge_p(path,file)
        index = int(np.log2(num_proc))
        result[0, :, index, :] = average_time
        result[1, :, index, :] = comm_time
        result[2, :, index, :] = dist_time
    return result

def generate_metrics_gescalapack(path):
    '''
    This function will generate 8x8x5 matrices for ge_scalapack
    1st dimension:problem size in thousand (1-8K)
    2nd dimension: number of processors-1 2 4 8 16 32 64 128
    3rd dimension: different independent runs'''
    #initialize result
    result = np.zeros((8, 8, 5), dtype = float)
    file_list = os.listdir(path)
    for file in file_list:
        num_proc, average_time = read_and_decode_ge_scalapack(path,file)
        index = int(np.log2(num_proc))
        result[:, index, :] = average_time
    return result

In [3]:
mc_dir = "../output/mc_p"
ge_dir = "../output/ge_p"
gescalapack_dir = "../output/ge_scalapack"
result_mc = generate_metrics_mcge_p(mc_dir)
result_ge = generate_metrics_mcge_p(ge_dir)
result_gescalapack = generate_metrics_gescalapack(gescalapack_dir)

In [4]:
#averaging 5 different runs
result_mc_average = np.mean(result_mc, axis = -1)
result_gep_average = np.mean(result_ge, axis = -1)
result_gescalapack_average = np.mean(result_gescalapack, axis = -1)

In [10]:
#Extract Serial time for all three algorithms (3x 8 matrix)
serial_time = np.array([result_mc_average[0,:,0], result_gep_average[0,:,0], result_gescalapack_average[:,0]])

#calculate the best serial time for the determinant task
best_serial_time = np.min(serial_time, axis= 0)

In [15]:
#calculate the speed up for all algorithms
speedup_mc = np.zeros((8,8), dtype = float)
speedup_ge = np.zeros((8,8), dtype = float)
speedup_gescalapack = np.zeros((8,8), dtype = float)

for i in range(8):
    t_s = best_serial_time[i]
    for j in range(8):
        speedup_mc[i,j] = t_s/ result_mc_average[0, i, j]
        speedup_ge[i,j] = t_s/ result_gep_average[0, i, j]
        speedup_gescalapack[i,j] = t_s/ result_gescalapack_average[i, j]

In [17]:
#calculate the average speedup for all problem size
average_speedup_mc = np.mean(speedup_mc, axis=0)
average_speedup_ge = np.mean(speedup_ge, axis=0)
average_speedup_gescalapack = np.mean(speedup_gescalapack, axis=0)

In [18]:
average_speedup_mc

array([ 0.9684402 ,  1.80587184,  3.78313625,  5.71478961,  7.21165183,
        9.03955253, 11.65056075,  9.33855536])

In [19]:
average_speedup_ge

array([1.        , 1.81138563, 3.73055726, 5.47046197, 6.55587263,
       4.36794431, 1.86292031, 0.82251768])

In [20]:
average_speedup_gescalapack

array([0.43910686, 0.68908454, 1.00344318, 1.21639564, 1.32301965,
       1.38455207, 1.40886772, 1.35528389])